## Creating and using a service user for Maxar Geospatial Platform
This sample workflow will guide a user through using the useres, tokens, and streaming classes of the Maxar Geospatial Platform SDK. The process will go through:
* Creating a new (service) user
* Setting permissions of the new (service) user
* Creating an API Key for the new (service) user
* Streaming imagery utilizing the API Key from the new (service) user

#### The following functionality is intended for account administrators and will not work with base user permissions

### Creating a new (service) user

#### Initialize the Maxar Geospatial Platform SDK Module

In [ ]:
from MGP_SDK.interface import Interface

admin_interface = Interface("<admin.user@email.address>", "<admin_user_password>", "<admin_user_client>")

#### Initialize the account-services class for Maxar Geospatial Platform SDK

In [ ]:
admin = admin_interface.account_service

In [ ]:
print(admin.get_self())

#### Create a new (service) user
In the cell below some variables will be pre-set to expedite the user creation process. *Note: If the account administrator has multiple activations under their account, the user will need to decide which activation the new (service) user will be tied to and obtain that activation's ID. If a user does not know the desired activation's ID, run the* `admin.get_self()` *function and print the result to view the activations available on the user's account.*

In [ ]:
user_type = "BASE_USER"
account_id = admin.get_self()['accountId']
# If multiple activations exist on an account, uncomment the activation_id variable below and comment out the
# activation_id variable below it and set the desired_activation_id in the list iteration.

# activation_id = [i['id'] for i in admin.get_self()['activations'] if i['id'] == <desired_activation_id>][0]
activation_id = [i['id'] for i in admin.get_self()['activations']][0]
email = "<new.user@email.address>"
first_name = "<NewUserFirstName>"
last_name = "<NewUserLastName>"
country = "<CountryOfOrigin>" #Example: US
client_id = "<adminUserClient>"
credit_limit = "<DesiredCreditLimit>"

new_user = admin.users.create_user(
    user_type=user_type, account_id=account_id, activation_id=activation_id, email_address=email, first_name=first_name, 
    last_name=last_name, country=country, client_id=client_id, dailyCreditLimitTotal=credit_limit
)
print(new_user)

#### Obtain desired permissions for new (service) user
This workflow will enable `VIVID` imagery for a service user. *Note:* `Roles` *and* `Permissions` *are used for the same terminology.*

The following cell will set the `clientContextId` to a variable. This is the actual ID associated with the user's client.

In [ ]:
client_context_id = [i['clientContextId'] for i in admin.get_self()['activations']][0]

The below cell will grab all non-ordering vivid roles and the WMS role and put the names of the roles into a list for use further on in the workflow.

In [ ]:
all_roles = admin.roles.get_roles(clientContextId=client_context_id)
desired_roles = [i['name'] for i in all_roles if "VIVID" in i['name'] and "ORDER" not in i['name'] or "WMS" in i['name']]
print(desired_roles)

### Setting permissions of the new (service) user
The new (service) user's ID will be needed for updating roles. The below cell will show a user how to obtain this.

In [ ]:
new_user_id = admin.users.get_users(username="<new.user's@email.address>")['userId']
print(new_user_id)

With the new (service) user's ID acquired, the roles for that user can now be updated.

In [ ]:
update_roles = admin.users.update_user_roles(user_id=new_user_id, roles_to_update=desired_roles, client_id=client_context_id)
print(update_roles)

The new (service) user now has the desired roles added to it and can stream Vivid imagery via the WMS method.

#### The following functionality can be utilized by base level users

### Creating an API Key for the new (service) user
With the user created, an API key can be generated for that user that can then be utilized to stream Vivid imagery.

#### Re-initialize the Maxar Geospatial Platform SDK Module with the new (service) user's credentials

In [ ]:
base_interface = Interface("<new.user's@email.address>", "<new_users_password>", "<adminUserClient>")

#### Initialize the token-services class for Maxar Geospatial Platform SDK

In [ ]:
tokens = base_interface.token

#### Create a new API Key

In [ ]:
new_token = tokens.create_token_record(name="Vivid Streaming", description="This token is used for streaming of Vivid imagery")
print(new_token)

From the above cell a user can visualize the `id`, `name`, `description`, `api_token` (key), and the `expiration_date` of the key, along with other information.

The below cell will set the `api_token` (key) of the token to a variable to be used further on in the workflow.

In [ ]:
new_token_id = new_token['api_token']
print(new_token_id)

### Streaming imagery utilizing the API Key from the new (service) user

This section of the workflow will assume that a new user wants to stream imagery but does not have a Maxar Geospatial Platform user. The user can stream imagery via the API key created earlier.

#### Declare variables for the streaming request
Below are some pre-filled variables that will expedite the streaming request.

In [ ]:
bbox = "39.911958,-105.006037,39.913785,-104.996821"
height = 256
width = 1024
layer = "Maxar:VividAdvanced"
crs = "EPSG:4326"
img_format = "image/vnd.jpeg-png"

#### Format URL

In [ ]:
streaming_url = f"https://api.maxar.com/streaming/v1/ogc/wms?service=WMS&request=GetMap&version=1.3.0&bbox={bbox}" \
                f"&crs={crs}&layers={layer}&height={height}&width={width}&format={img_format}&maxar_api_token={new_token_id}"

#### Import the requests module and the IPython module
If the `requests` and/or `IPython` modules are not currently installed, be sure to install them via `pip`. *Note: The* `IPython` *module is not required for other use cases, but enables imagery visualization in Jupyter Notebooks.*

In [ ]:
import requests
import IPython.display as disp
from IPython.display import Image, display

With the requests module installed, a user can now pass in the formatted url and visualize the imagery.

In [ ]:
response = requests.request("GET", streaming_url)
display(disp.Image(response.content))